In [1]:
# !conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia


In [1]:
# !pip install matplotlib

In [1]:
from sklearn.metrics import roc_auc_score

In [2]:
import torch

In [3]:
import torch.nn as nn

In [4]:
torch.cuda.is_available()

True

In [5]:
x = torch.rand(5, 3)
print(x)

tensor([[0.7209, 0.9404, 0.5645],
        [0.3859, 0.4065, 0.2243],
        [0.7746, 0.4664, 0.1597],
        [0.1633, 0.9118, 0.7845],
        [0.3802, 0.6604, 0.3582]])


In [6]:
import pandas as pd

In [7]:
import numpy as np

import torch
from torch import nn
# from torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing


In [8]:
import matplotlib.pyplot as plt

In [9]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [10]:
class MyDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X.astype(np.float32))
        self.y = torch.from_numpy(y.astype(np.float32))
        
    def __getitem__(self, index):
        return self.X[index], self.y[index]
    
    def __len__(self):
        return self.X.shape[0]
    
#     def input_size(self):
#         return self.X.shape[1]

In [38]:
class CreditNN(nn.Module):
    def __init__(self, input_size=411, hidden_1_size=1000, out_size=1):
        super(CreditNN, self).__init__()
        self.to(device)
        self.input_size = input_size
        self.dense1 = nn.Linear(self.input_size, hidden_1_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.dense2 = nn.Linear(hidden_1_size, out_size)
#         self.normalize = nn.BatchNorm1d(hidden_1_size)
#         self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=1)
        self.to(device)
    
    def forward(self, x):
        x = self.dense1(x)
        x = self.relu(x)
#         x = x.view(-1, self.dense1.out_features)
#         x = self.normalize(x)
#         x = self.dropout(x)
        x = self.dense2(x)
#         x = self.sigmoid(x)
#         x = self.softmax(x)
        return x
    
    def fit(self, data_loader, optimizer, criterion, epochs=10, verbose=True):
        self.__init__(input_size=data_loader.dataset.X.shape[1])
        print(self.input_size)
        for epoch in range(epochs):
            for x_batch, y_batch in data_loader:
                y_batch = y_batch.to(device)
                y_pred = self(x_batch.to(device))
                loss = criterion(y_batch, y_pred[:, 0])
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            if verbose: 
                print(f'Epoch #{epoch}: \tloss: {loss.item():.5f}')
                print(y_batch)
                print(y_pred[:, 0])
#                 print(loss.item())
        print(f'Training loss: {loss.item():.4f}')
        
    def predict(self, X):
        return self(torch.from_numpy(X.astype(np.float32)).to(device))

In [39]:
my_nn = CreditNN()

In [13]:
train = pd.read_pickle('clean-train-5.pkl')
test = pd.read_pickle('clean-test-5.pkl')

In [14]:
target = 'flag'
features = train.drop(columns=target).columns.tolist()

In [15]:
for col in train.columns:
    if train[col].dtype == bool:
        train[col] = train[col].astype(np.int8)
        test[col] = test[col].astype(np.int8)

In [16]:
X = train[features]
y = train[target]

In [40]:
loss_fn = nn.MSELoss()
# loss_fn = nn.BCELoss()
# loss_fn = nn.CrossEntropyLoss()


optimizer = torch.optim.Adam(my_nn.parameters(), lr=0.000001)

In [41]:
my_nn.to(device)

CreditNN(
  (dense1): Linear(in_features=411, out_features=1000, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (dense2): Linear(in_features=1000, out_features=1, bias=True)
  (softmax): Softmax(dim=1)
)

In [19]:
y.values.shape

(2100000,)

In [20]:
train_data = MyDataset(X.values, y.values)

In [28]:
batch_size = 3_000

train_dataloader = DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True
)

In [22]:
train_dataloader.dataset.X.shape

torch.Size([2100000, 411])

In [42]:
%%time
my_nn.fit(train_dataloader, optimizer, loss_fn)

411
Epoch #0: 	loss: 116.39467
tensor([0., 0., 1.,  ..., 0., 0., 0.], device='cuda:0')
tensor([11.0980, 12.0988,  9.7547,  ..., 12.1003,  9.9975, 11.6913],
       device='cuda:0', grad_fn=<SelectBackward0>)
Epoch #1: 	loss: 116.26477
tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0')
tensor([ 7.8907,  6.5455, 12.4232,  ..., 10.0564, 10.8875,  7.9546],
       device='cuda:0', grad_fn=<SelectBackward0>)
Epoch #2: 	loss: 117.20963
tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0')
tensor([ 6.2683,  9.9263, 12.1005,  ...,  9.3983,  9.4393, 12.3862],
       device='cuda:0', grad_fn=<SelectBackward0>)
Epoch #3: 	loss: 117.73159
tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0')
tensor([ 7.5290, 11.6397, 12.5696,  ..., 10.8295, 17.5697,  9.8097],
       device='cuda:0', grad_fn=<SelectBackward0>)
Epoch #4: 	loss: 117.52104
tensor([0., 0., 0.,  ..., 0., 1., 0.], device='cuda:0')
tensor([12.2983,  8.1309,  8.5450,  ...,  9.9426, 10.8708, 10.7213],
       device='cuda:0', grad_fn=

In [43]:
lss_pred = my_nn(torch.from_numpy(test[features].values.astype(np.float32)).to(device))


In [44]:
pred = lss_pred.to('cpu').detach().numpy()

In [45]:
roc_auc_score(test[target], pred)

0.5356464384658775

In [ ]:
from torch.utils.data import DataLoader

batch_size = 64

train_dataloader = DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True
)

In [ ]:
import torch.nn as nn

class MyNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        
        self.hidden = nn.Linear(input_dim, hidden_dim)
        
        self.f1 = nn.ReLU()
        
        self.output = nn.Linear(hidden_dim, output_dim)
        
        self.f2 = nn.Sigmoid()
        
    def forward(self, x):
        x = self.f1(self.hidden(x))
        x = self.f2(self.output(x))
        
        return x

In [ ]:
my_net = MyNet(2, 16, 1)

print(my_net)

In [ ]:
y_pred = my_net(torch.from_numpy(X_test.astype(np.float32)))

In [ ]:
loss_fn = nn.BCELoss()

optimizer = torch.optim.SGD(my_net.parameters(), lr=0.1)

In [ ]:
num_epochs = 50

loss_values = []

for epoch in range(num_epochs):
    for X, y in train_dataloader:
        
        pred = my_net(X)
        
        loss = loss_fn(pred, y.unsqueeze(-1))
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
        
    loss = loss_fn(pred, y.unsqueeze(-1))
    loss_values.append(loss.item())

In [ ]:
step = np.linspace(0, num_epochs)

fig, ax = plt.subplots(figsize=(8,5))
plt.plot(step, np.array(loss_values))
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()